# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 34.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [8]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [9]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [10]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city not just because it's the seat of the federal government, but also because it's rich in history and culture. If you ever have the chance to visit, the National Mall, museums, and historical monuments are definitely worth exploring.


Add assistant messages to teach LLM what `##` is.

In [11]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [12]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [13]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [14]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [15]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])

print(tweet_data)

['RT @Artemisfornow: GLOBALISM - Every leader who dares challenge the establishment meets the same fate, prosecution, imprisonment, or exile.…', 'RT @MrPitbull07: @MLP_officiel When democracy fails to uphold its own values, it morphs into tyranny disguised as justice.\n\nMarine Le Pen w…', 'RT @morandiniblog: URGENT - Marine Le Pen sur TF1 : "Je n\'ai pas confiance dans la date de l\'appel parce que ce n\'est pas moi qui en est la…', 'RT @HansMahncke: The Marine Le Pen witch hunt is even worse than you think. What the media is breathlessly calling “embezzlement” is nothin…', "RT @tavoliakys: MARINE LE PEN ISN'T ELIGIBLE FOR THE FRENCH PRESIDENTIAL ELECTIONS https://t.co/3HofIZlOWm", 'RT @bennyjohnson: Right-Wing French Politician Marine Le Pen has been convicted of embezzlement. \n\nShe can’t run for office for five years…', 'RT @Inevitablewest: 🚨BREAKING: Italian Deputy PM:\n\n"Banning Marine Le Pen is a declaration of war from Brussels."\n\nThe EU don’t know what’s…', 'RT @PGKroeger:

In [16]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  100


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [17]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss the recent conviction of French far-right leader Marine Le Pen for embezzling European Union funds, resulting in a four-year prison sentence and a five-year ban from holding public office. The decision has sparked significant controversy and debate, with some viewing it as a political move against right-wing figures. Various international figures, including Russia and Italy's Deputy Prime Minister, have criticized the ruling, claiming it undermines democratic norms. Supporters of Le Pen argue that the verdict is unjust and politically motivated, while others emphasize the importance of accountability for financial misconduct.


In [18]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

Marine Le Pen convicted of embezzlement, banned from elections, sparking international reactions and claims of political motives.


In [20]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss motivation,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss motivation in the context of political actions and consequences, highlighting how leaders like Marine Le Pen face legal challenges when opposing the establishment. The narrative suggests that such actions are politically motivated, aiming to suppress dissent and maintain control, sparking debates on democracy and justice.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [21]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [22]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
RT @Benjam1Lucas: Marine Le Pen est une délinquante.
===
harassment
RT @PatrioteEngager: #RN  C est cette Immonde femme la juge #Macroniste Bénédicte de Perthuis qui a rendu #ineligible  Marine Le Pen  Cette…
===
harassment
RT @LeLapinDuFutur: Marine Le Pen inéligible le lendemain de la fin du ramadan où l'on a pu voir des millions de musulmans prier dans la ru…
===
harassment
RT @Benramine_: Phillipe Poutou ce visionnaire il avait raison sur Fillion , sur Marine Le pen quel goat https://t.co/FEeEs85w0i
===
harassment
RT @FredGaulois: Gilles Bouleau face à  Marine Le Pen est juste insupportable, on voit bien toute sa haine, alors qu il ferme sa gueule fac…


### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [23]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

RT @Artemisfornow: 全球主义——每一个敢于挑战体制的领导人都会面临相同的命运：起诉、监禁或流亡。…
RT @MrPitbull07: @MLP_officiel 当民主未能坚持其自身价值时，它就会变成伪装成正义的暴政。

玛丽娜·勒庞 w…
RT @morandiniblog：紧急 - 玛丽娜·勒庞在TF1上说：“我不相信呼叫的日期，因为这不是我决定的……”
RT @HansMahncke: 对玛丽娜·勒庞的政治迫害比你想象的还要严重。媒体急切地称之为“挪用公款”的事情其实根本不是这样……
转发 @tavoliakys：玛丽娜·勒庞不符合法国总统选举的资格 https://t.co/3HofIZlOWm
RT @bennyjohnson: 右翼法国政治家玛丽娜·勒庞因贪污罪被判有罪。

她五年内不能参选……
RT @Inevitablewest: 🚨突发新闻：意大利副总理：

“禁止玛丽娜·勒庞是布鲁塞尔的宣战声明。”

欧盟不知道什么是……
RT @PGKroeger: 勒庞要求对那些在财务上作弊的政客进行终身排除 🇫🇷🇪🇺💶🤪 https://t.co/UrXJE6W6Te
RT @Ilangabet: 线索：玛丽娜·勒庞被判处4年监禁和5年不得参选，并立即执行。

国民阵线将发起一项……
RT @EvaVlaar: 针对欧洲右翼的法律战正在被提升到一个全新的水平。

玛丽娜·勒庞 @MLP_officiel 已被判决…
RT @BRICSinfo: 最新消息：🇷🇺🇫🇷 俄罗斯称对玛丽娜·勒庞的定罪“违反了民主规范。” https://t.co/tT8NEUd7zZ
RT @AdresanSoso: 玛丽娜·勒庞可能面临监禁和丧失参选资格，因为她涉嫌从欧盟挪用400万欧元。…
RT @stillgray: 玛丽娜·勒庞刚刚被判处两年监禁。这太疯狂了。https://t.co/l71TDXQZ4j
RT @CilComLFC: 法国人民热爱玛丽娜·勒庞，并要求立即推翻她的“有罪”判决！🇫🇷🇫🇷🇫🇷 https://t.co/ct…
RT @PopCrave: 法国极右翼领导人玛丽娜·勒庞被判定挪用欧盟资金罪名成立。https://t.co/DlMrZS5ZQi
RT @La_SER: 🔴 最新消息 | 法国极右翼的玛丽娜·勒庞和其他8名欧洲议会议

In [24]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Ah, yes, GLOBALISM, that delightful little game where any leader with the audacity to ruffle the feathers of the establishment finds themselves in a rather unfortunate predicament—prosecution, imprisonment, or perhaps a charming little exile. How utterly predictable.
RT @MrPitbull07: @MLP_officiel Oh, the irony! When democracy, that supposed bastion of freedom, decides to play dress-up as tyranny, all in the name of justice. How delightfully absurd! Marine Le Pen w…
RT @morandiniblog: Oh, how utterly delightful! Marine Le Pen on TF1: "I simply cannot trust the date of the appeal, for it is not I who holds the reins of such matters..."
RT @HansMahncke: Oh, the audacity! The Marine Le Pen witch hunt is even more ghastly than you could possibly imagine. What the media, in their infinite wisdom, is dramatically labeling as “embezzlement” is, in fact, a mere trifle...
Oh, how delightfully amusing! It appears Marine Le Pen has been deemed unfit for the French presidential elections. Quelle s

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [25]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["Artemisfornow"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["@MrPitbull07", "@MLP_officiel", "Marine Le Pen"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "distrust",
  "mentioned": ["Marine Le Pen"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["HansMahncke", "Marine Le Pen"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["tavoliakys", "Marine Le Pen"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "mentioned": ["bennyjohnson", "Marine Le Pen"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "anger",
  "mentioned": ["Inevitablewest", "Marine Le Pen"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "indifference",
  "mentioned": ["PGKroeger", "Le P

In [26]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "1": "Marine Le Pen's Conviction",
  "2": "Embezzlement Charges",
  "3": "Political Ineligibility",
  "4": "Reactions from International Leaders",
  "5": "Impact on French Politics",
  "6": "Criticism of Judicial System",
  "7": "Globalism and Political Persecution",
  "8": "Support for Marine Le Pen",
  "9": "Media Coverage and Public Opinion",
  "10": "Comparisons to Other Political Figures"
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [27]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 100/100 [01:52<00:00,  1.13s/it]


In [28]:
print(analysis_result)

['{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["Artemisfornow"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["MrPitbull07", "MLP_officiel", "Marine Le Pen"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "distrust",\n  "mentioned": ["Marine Le Pen"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["HansMahncke", "Marine Le Pen"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["tavoliakys", "Marine Le Pen"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "informative",\n  "mentioned": ["bennyjohnson", "Marine Le Pen"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "anger",\n  "mentioned": ["Inevitablewest", "Marine Le Pen"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotio

In [29]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 2,
  "Republican count": 23,
  "people name": {
    "Artemisfornow": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "MrPitbull07": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "MLP_officiel": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "Marine Le Pen": {
      "sentiments": ["negative", "neutral", "positive"],
      "emotions": ["frustration", "distrust", "informative", "anger", "indifference", "concern", "outrage", "disapproval", "rejection", "defiance", "support", "surprise", "curiosity", "sarcasm", "trust", "shock", "suspicion", "admiration"]
    },
    "HansMahncke": {
      "sentiments": ["negative"],
      "emotions": ["frustration"]
    },
    "tavoliakys": {
      "sentiments": ["neutral"],
      "emotions": ["informative"]
    },
    "bennyjohnson": {
      "sentiments": ["neutral"],
      "emotions": ["informative"]
    },
    "Inevitablewest": {


## Create a chatbot

In [30]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

You:  thanks


Chatbot: You're welcome! If you have any other questions or need further information, feel free to ask.


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
